In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
from tpot import TPOTClassifier

# --- 1. Tải và TIỀN XỬ LÝ dữ liệu ---
# TPOT YÊU CẦU DỮ LIỆU SẠCH VÀ Ở DẠNG SỐ

# Tải bộ dữ liệu Titanic
data = sns.load_dataset('titanic')

# Chỉ lấy các cột cơ bản giống như ví dụ trước
data = data[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']]

# **BƯỚC TIỀN XỬ LÝ BẮT BUỘC CHO TPOT**
# 1. Xử lý giá trị bị thiếu (NaNs)
# Điền 'age' (tuổi) bằng giá trị trung vị
data['age'] = data['age'].fillna(data['age'].median())
# Điền 'embarked' (cảng lên tàu) bằng giá trị xuất hiện nhiều nhất
data['embarked'] = data['embarked'].fillna(data['embarked'].mode()[0])

# 2. Mã hóa One-Hot cho các biến categorical
# Chuyển 'sex' và 'embarked' thành các cột số (0/1)
# drop_first=True để tránh bẫy đa cộng tuyến
data_processed = pd.get_dummies(data, columns=['sex', 'embarked'], drop_first=True)

# 3. Tách X (features) và y (target)
X = data_processed.drop('survived', axis=1)
y = data_processed['survived']

# Chia dữ liệu: 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    stratify=y,
                                                    random_state=42)

print(f"Kích thước tập Train (đã xử lý): {X_train.shape}")
print(f"Kích thước tập Test (đã xử lý): {X_test.shape}")


# --- 2. Thiết lập và Huấn luyện (Train) TPOT ---

# Khởi tạo TPOTClassifier
# generations: Số thế hệ "tiến hóa". Càng cao càng tốt nhưng càng lâu. (5-10 cho demo)
# population_size: Số lượng pipeline trong mỗi thế hệ. (20-50 cho demo)
# scoring: Thước đo để tối ưu, giống LAMA ta dùng 'roc_auc'
# verbosity: 2 để hiển thị tiến trình
# n_jobs: -1 để dùng tất cả CPU
tpot = TPOTClassifier(generations=5, 
                      population_size=20, 
                      scoring='roc_auc',
                      verbosity=2, 
                      random_state=42, 
                      n_jobs=-1,
                      cv=5) # Sử dụng 5-fold Cross-Validation

# Huấn luyện mô hình
# CẢNH BÁO: Bước này sẽ mất thời gian (vài phút đến vài giờ)
# tùy thuộc vào 'generations' và 'population_size'.
print("\nBắt đầu huấn luyện TPOT (Quá trình này sẽ mất vài phút)...")
tpot.fit(X_train, y_train)


# --- 3. Đánh giá (Evaluate) trên tập Test ---

print("\nĐã huấn luyện xong. Bắt đầu dự đoán trên tập Test...")

# Dự đoán xác suất
test_probs_class_1 = tpot.predict_proba(X_test)[:, 1]

# Dự đoán nhãn (0 hoặc 1)
y_pred = tpot.predict(X_test)

# In kết quả
auc_score = roc_auc_score(y_test, test_probs_class_1)
acc_score = accuracy_score(y_test, y_pred)

print("--- KẾT QUẢ ĐÁNH GIÁ (TPOT) ---")
print(f"ROC-AUC trên tập Test: {auc_score:.4f}")
print(f"Accuracy trên tập Test: {acc_score:.4f}")


# --- 4. (Quan trọng) Xuất ra pipeline tốt nhất ---
# Đây là tính năng mạnh nhất của TPOT: nó cho bạn code Python
# của pipeline mà nó tìm thấy.
print(f"\nPipeline tốt nhất được tìm thấy (điểm CV: {tpot.score(X_train, y_train):.4f}):")
# print(tpot.fitted_pipeline_) # In ra cấu trúc pipeline

# Xuất pipeline ra file .py để bạn có thể dùng sau này
tpot.export('tpot_titanic_pipeline.py')
print("Đã xuất pipeline tốt nhất ra file 'tpot_titanic_pipeline.py'")